Task 01 B

In [1]:
class UtilityBasedAgent:
    def __init__(self, goal):
        self.goal = goal


    def formulate_goal(self, percept):
        if percept == self.goal:
            return "Goal reached"
        return "Searching"

    def ucs_search(self, graph, start, goal):
        frontier = [(start, 0)]
        visited = set()
        cost_so_far = {start: 0}
        came_from = {start: None}

        while frontier:
            frontier.sort(key=lambda x: x[1])
            current_node, current_cost = frontier.pop(0)

            if current_node in visited:
                continue

            visited.add(current_node)

            if current_node == goal:
                path = []
                while current_node is not None:
                    path.append(current_node)
                    current_node = came_from[current_node]
                path.reverse()
                return f"Goal found with UCS. Path: {path}, Total Cost: {current_cost}"

            for neighbor, cost in graph[current_node].items():
                new_cost = current_cost + cost
                if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                    cost_so_far[neighbor] = new_cost
                    came_from[neighbor] = current_node
                    frontier.append((neighbor, new_cost))

        return "Goal not found"

    def act(self, percept, graph):
        goal_status = self.formulate_goal(percept)
        if goal_status == "Goal reached":
            return f"Goal {self.goal} found!"
        else:
            return self.ucs_search(graph, percept, self.goal)


class Environment:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node


def run_agent(agent, environment, start_node):
    percept = environment.get_percept(start_node)
    action = agent.act(percept, environment.graph)
    print(action)


graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'D': 2, 'E': 5},
    'C': {'A': 4, 'D': 3},
    'D': {'B': 2, 'C': 3, 'E': 1},
    'E': {'B': 5, 'D': 1}
}

environment = Environment(graph)
agent = UtilityBasedAgent(goal='C')

run_agent(agent, environment, start_node='A')


Goal found with UCS. Path: ['A', 'C'], Total Cost: 4


Task 01 B

In [2]:
class GoalBasedAgent:
    def __init__(self, goal, depth_limit):
        self.goal = goal
        self.depth_limit = depth_limit

    def formulate_goal(self, percept):
        if percept == self.goal:
            return "Goal reached"
        return "Searching"

    def dls_search(self, graph, start, goal, depth_limit):
        visited = []

        def dfs(node, depth):
            if depth > depth_limit:
                return None
            visited.append(node)
            if node == goal:
                return visited

            for neighbor in graph.get(node, []):
                if neighbor not in visited:
                    path = dfs(neighbor, depth + 1)
                    if path:
                        return path

            visited.pop()
            return None

        return dfs(start, 0)

    def act(self, percept, graph):
        goal_status = self.formulate_goal(percept)
        if goal_status == "Goal reached":
            return f"Goal {self.goal} found!"
        else:
            path = self.dls_search(graph, percept, self.goal, self.depth_limit)
            if path:
                return f"Goal found with DLS. Path: {path}"
            else:
                return "Goal not found within the depth limit"


class Environment:
    def __init__(self, graph):
        self.graph = graph

    def get_percept(self, node):
        return node


def run_agent(agent, environment, start_node):
    percept = environment.get_percept(start_node)
    action = agent.act(percept, environment.graph)
    print(action)


graph = {
    'A': ['B', 'C'],
    'B': ['A', 'D', 'E'],
    'C': ['A', 'F'],
    'D': ['B'],
    'E': ['B', 'G'],
    'F': ['C'],
    'G': ['E']
}


environment = Environment(graph)
agent = GoalBasedAgent(goal='G', depth_limit=3)
run_agent(agent, environment, start_node='A')


Goal found with DLS. Path: ['A', 'B', 'E', 'G']


Task 02

In [3]:

graph = {
    '1': {'2': 10, '4': 20, '3': 15},
    '2': {'1': 10, '4': 25, '3': 35},
    '3': {'1': 15, '4': 30, '2': 35},
    '4': {'1': 20, '2': 25, '3': 30}
}


def travelling_salesman_problem(graph, start_node):
    nodes = [node for node in graph if node != start_node]
    min_cost = float('inf')
    best_path = []


    def backtrack(current_node, visited, current_cost, path):
        nonlocal min_cost, best_path


        if len(visited) == len(nodes):
            return_cost = graph[current_node].get(start_node, float('inf'))
            total_cost = current_cost + return_cost


            if total_cost < min_cost:
                min_cost = total_cost
                best_path = path + [start_node]
            return

        for next_node in nodes:
            if next_node not in visited:
                visited.add(next_node)
                backtrack(next_node, visited, current_cost + graph[current_node].get(next_node, float('inf')), path + [next_node])
                visited.remove(next_node)

    backtrack(start_node, {start_node}, 0, [start_node])

    return min_cost, best_path


start_node = '1'


minimum_path_cost, path_taken = travelling_salesman_problem(graph, start_node)
print(f"Minimum Cost: {minimum_path_cost}")
print(f"Path Taken: {' -> '.join(path_taken)}")

Minimum Cost: 55
Path Taken: 1 -> 2 -> 4 -> 1


Task 03

In [4]:
import heapq

def uniform_cost_search(graph, start, goal):
    priority_queue = [(0, start, [start])]
    visited = set()

    while priority_queue:
        cost, current_node, path = heapq.heappop(priority_queue)

        if current_node == goal:
            return path, cost

        if current_node in visited:
            continue

        visited.add(current_node)

        for neighbor, edge_cost in graph[current_node]:
            if neighbor not in visited:
                new_cost = cost + edge_cost
                new_path = path + [neighbor]
                heapq.heappush(priority_queue, (new_cost, neighbor, new_path))

    return None, float('inf')

graph = {
    'A': [('B', 5), ('C', 2)],
    'B': [('D', 1)],
    'C': [('D', 4)],
    'D': []
}

start_node = 'A'
goal_node = 'D'
path, cost = uniform_cost_search(graph, start_node, goal_node)
print(f"Optimal Path: {path}, Total Cost: {cost}")

Optimal Path: ['A', 'B', 'D'], Total Cost: 6


Task 04

In [5]:
from collections import deque

def is_goal(state):
    return state == [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

def find_empty_tile(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

def generate_moves(state):
    moves = []
    i, j = find_empty_tile(state)
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for di, dj in directions:
        ni, nj = i + di, j + dj
        if 0 <= ni < 3 and 0 <= nj < 3:
            new_state = [row[:] for row in state]
            new_state[i][j], new_state[ni][nj] = new_state[ni][nj], new_state[i][j]
            moves.append(new_state)
    return moves

def depth_limited_search(state, goal, depth_limit):
    stack = [(state, 0, [])]
    visited = set()

    while stack:
        current_state, current_depth, path = stack.pop()

        print(f"Exploring State (Depth {current_depth}):")
        for row in current_state:
            print(row)
        print()

        if current_depth > depth_limit:
            print("Depth limit exceeded. Backtracking...\n")
            continue

        if tuple(map(tuple, current_state)) in visited:
            print("State already visited. Skipping...\n")
            continue

        visited.add(tuple(map(tuple, current_state)))

        if is_goal(current_state):
            print("Goal State Found!")
            print("Solution Path:")
            for step in path + [current_state]:
                for row in step:
                    print(row)
                print()
            return True

        if current_depth < depth_limit:
            for move in generate_moves(current_state):
                stack.append((move, current_depth + 1, path + [current_state]))

    return False

initial_state = [
    [1, 2, 3],
    [4, 5, 6],
    [7, 0, 8]
]

depth_limit = 10
solution_found = depth_limited_search(initial_state, None, depth_limit)
if not solution_found:
    print("No solution found within the depth limit.")

Exploring State (Depth 0):
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

Exploring State (Depth 1):
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Goal State Found!
Solution Path:
[1, 2, 3]
[4, 5, 6]
[7, 0, 8]

[1, 2, 3]
[4, 5, 6]
[7, 8, 0]



Task 05

In [6]:
class DecisionNode:
    def __init__(self, state, path=None):
        self.state = state
        self.path = path if path else []

    def __repr__(self):
        return f"Tasks left: {self.state['tasks']}, Resources: {self.state['resources']}, Path: {self.path}"

def is_goal(state):
    return len(state["tasks"]) == 0 and state["resources"] >= 0

def generate_choices(node):
    choices = []
    for task in node.state["tasks"]:
        new_state = {
            "tasks": [t for t in node.state["tasks"] if t != task],
            "resources": node.state["resources"] - 5
        }
        choices.append(DecisionNode(new_state, node.path + [task]))
    return choices

def depth_first_search(node, max_depth=10):
    print(f"STEP {len(node.path)}: Resources={node.state['resources']}, " +
          f"Tasks left={node.state['tasks']}, Path so far={node.path}")

    if is_goal(node.state):
        print("\nGOAL REACHED!")
        return node.path

    if len(node.path) >= max_depth:
        print(f"  (Reached max depth of {max_depth}, backtracking...)")
        return None

    for next_node in generate_choices(node):
        print(f"  Trying: {next_node.path[-1]}")
        result = depth_first_search(next_node, max_depth)
        if result:
            return result

    print(f"  (No valid paths from here, backtracking...)")
    return None

initial_state = {
    "tasks": ["task1", "task2", "task3"],
    "resources": 20
}


initial_node = DecisionNode(initial_state)
solution = depth_first_search(initial_node)

if solution:
    print("\nFINAL SOLUTION:")
    resources_left = initial_state["resources"]
    for i, task in enumerate(solution):
        resources_left -= 5
        print(f"Step {i+1}: Complete {task} (Resources remaining: {resources_left})")
else:
    print("\nNo solution found!")

STEP 0: Resources=20, Tasks left=['task1', 'task2', 'task3'], Path so far=[]
  Trying: task1
STEP 1: Resources=15, Tasks left=['task2', 'task3'], Path so far=['task1']
  Trying: task2
STEP 2: Resources=10, Tasks left=['task3'], Path so far=['task1', 'task2']
  Trying: task3
STEP 3: Resources=5, Tasks left=[], Path so far=['task1', 'task2', 'task3']

GOAL REACHED!

FINAL SOLUTION:
Step 1: Complete task1 (Resources remaining: 15)
Step 2: Complete task2 (Resources remaining: 10)
Step 3: Complete task3 (Resources remaining: 5)


Task 06

In [7]:
def is_valid(board, row, col, num):
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False

    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for i in range(start_row, start_row + 3):
        for j in range(start_col, start_col + 3):
            if board[i][j] == num:
                return False

    return True

def solve_sudoku(board):
    for row in range(9):
        for col in range(9):
            if board[row][col] == 0:
                for num in range(1, 10):
                    if is_valid(board, row, col, num):
                        board[row][col] = num

                        if solve_sudoku(board):
                            return True

                        board[row][col] = 0

                return False
    return True

sudoku_board = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Solve Sudoku
if solve_sudoku(sudoku_board):
    for row in sudoku_board:
        print(row)
else:
    print("No solution exists")

[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
